In [1]:
import pandas
import requests
from GIS_Data_Library_ext import *
import os

In [2]:
oe_api_key = os.environ['OE_API_KEY']

# Bring in data from OE databanks
Find saved databank selections

In [3]:
gem_selection = find_selection(oe_api_key, 'PPI_GEM_Q')
nac_selection = find_selection(oe_api_key, 'PPI_NAC_Q')
gis_selection = find_selection(oe_api_key, 'PPI_GIS_Q')
usind_selection = find_selection(oe_api_key, 'PPI_USIND_Q')

In [4]:
selections = [gem_selection, nac_selection, gis_selection, usind_selection]
databanks = ['gem', 'nac', 'gis', 'usind']

In [5]:
selection_dict = dict(zip(databanks, selections))

Download databank selections

In [8]:
data_main = {}
metadata_main = {}

for i in selection_dict.keys():
    # Pull Global Economic Service data
    response = requests.get("https://services.oxfordeconomics.com/api/download/" + selection_dict[i],
                            headers = {'Accept' : 'application/json', 
                                        'API-Key' : oe_api_key,
                                        'Content-Type' : 'application/json; charset=utf8'})

    data_pull = response.json()

    # extract mnemonics, quarterly data and metadata
    data = {}
    metadata = {}
    for j in range(0, len(data_pull)):
        vbl = data_pull[j]['VariableCode']
        data_ts = data_pull[j]['QuarterlyData']
        metadata = data_pull[j]['Metadata']
        data[vbl] = data_ts
        metadata[vbl] = metadata

    data_main[i] = data
    metadata[i] = metadata

Convert to dataframes

In [11]:
dataframes = {}

In [29]:
for i in data_main.keys():
    dataframes[i] = pd.DataFrame(data_main[i], index = pd.period_range(start='1980-01-01', periods = len(list(data_main['nac'].values())[0]), freq='Q'))